## ReMKiT1D input generator - electron-ion collision operator test for l=0

This example tests temperature relaxation between electrons and ions due to Coulomb collisions.

This test corresponds to the second test in Section 6.2.2. in the ReMKiT1D paper.


In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.common_models as cm
import RMK_support.sk_normalization as skn

import scipy.optimize

### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for IO files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_ei_0_coll_test/" 
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

#### MPI


In [ ]:
numProcsX = 1 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic 
numProcs = numProcsX * numProcsH
haloWidth = 1 # Halo width in cells

rk.setMPIData(numProcsX,numProcsH,haloWidth)

#### PETSc

Default settings are used here.

#### HDF5

No input HDF5 file is used here and all variables are outputted by default using the wrapper class.

### Normalization setup

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

timeNorm = skn.collTimeei(rk.normalization["eVTemperature"],rk.normalization["density"],rk.normalization["referenceIonZ"])

### Grid setup

In [ ]:
xGrid = np.ones(1) # 0D
dv0 = 0.0307
cv = 1.025
vGrid = [dv0/2]
for i in range(1,120):
    vGrid.append(vGrid[i-1]*cv)
lMax = 0 
gridObj = Grid(xGrid,np.array(vGrid),lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True)

In [ ]:
# Add the grid to the wrapper
rk.grid=gridObj

### Set temperature derivation option and add electron species

In [ ]:
rk.setStandardTextbookOptions([-1,0]) 

rk.addSpecies("e",0)
rk.addSpecies("D+",-1,atomicA=2.014,charge=1.0)

### Variable initialization

In [ ]:
T0 = 0.8
n0 = 1.0
T = T0*np.ones(gridObj.numX())
n = n0 * np.ones(gridObj.numX())
W = 3*n*T/2

f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = (T0*np.pi)**(-1.5) * n0 * np.exp(-gridObj.vGrid**2/T0) 
    
rk.addVar("f",f,isDistribution=True)
rk.addVar("Wi",W/2,units='$10eV$')
rk.addVar("W",W,units='$10eV$',isDerived=True,derivationRule=sc.derivationRule("energyMoment",["f"]))
rk.addVar("n",n,units='$10^{19} m^{-3}$',isDerived=True,derivationRule=sc.derivationRule("densityMoment",["f"]))
rk.addVar("zeroVar",isDerived=True,outputVar=False)
rk.addVar("T",T,isDerived=True,derivationRule=sc.derivationRule("tempFromEnergye",["W","n","zeroVar"]))
rk.addVar("Ti",T/2,isDerived=True,derivationRule=sc.derivationRule("tempFromEnergyD+",["Wi","n","zeroVar"]))
rk.addVar("time",isScalar=True,isDerived=True)

### Adding e-e collision operator model for l = 0

Adding the electron-electron operator here to keep the distribution from deviating from a Maxwellian for the analytical comparison

In [ ]:
cm.addEECollIsotropic(modelTag="e-e0",distFunName="f",elTempVar="T",elDensVar="n",wrapper=rk)


### Adding e-i collision operator model for l = 0

The e-i collision operator for l=0 is implemented in common_models.py and only used here.

In [ ]:
cm.addEICollIsotropic(modelTag="e-i0",distFunName="f",elTempVar="T",elDensVar="n",ionTempVar="Ti",ionDensVar="n",ionSpeciesName="D+",wrapper=rk,ionEnVar="Wi")

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = sc.picardBDEIntegrator(absTol=10.0) #Everything default except for more lenient absolute convergence tolerance

rk.addIntegrator("BE",integrator)

Set initial timestep length and numbers of allowed implicit and general groups

In [ ]:
initialTimestep =  0.1

rk.setIntegratorGlobalData(1,1,initialTimestep) 

Single integration step

In [ ]:
bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("BE1",bdeStep.dict())

Adding Coulomb log diagnostic variable using the extractor manipulator.

In [ ]:
rk.addVar("logLei",isDerived=True)
rk.addManipulator("logLeiExtractor",sc.extractorManipulator("e-i0","logLei","logLei",priority=4))

#### Timeloop options

In [ ]:
rk.setFixedNumTimesteps(40000)
rk.setFixedStepOutput(500)

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


In [ ]:
numFiles = 80

#### Loading data

Set loadpath to ReMKiT1D directory

In [ ]:
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

In [ ]:
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames,varsToIgnore="zeroVar")
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=80,dpi=150)

### Compare with analytical solution

Note that here we let the Coulomb log vary, so that might introduce another small error compared to the analytical solution. For analytical solution form see Shkarofsky's book Chapter 7-10.

In [ ]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
epsilon0 = 8.854188e-12 #vacuum permittivity 
amu = 1.6605390666e-27 #atomic mass unit
gamma0norm = elCharge**4/(4*np.pi*elMass**2*epsilon0**2)
Ttot = (loadedData["T"].data[0,0] + loadedData["Ti"].data[0,0])/2 # Plasma temperature
tei0 = 8*gamma0norm  * elMass / (rk.getSpecies("D+").atomicA * amu)*2*loadedData["n"].data[0]*rk.normalization["density"]*(elMass/(2*elCharge*Ttot*rk.normalization["eVTemperature"]))**(3/2)/(3*np.sqrt(np.pi)) 

In [ ]:
norms = skn.calculateNorms(10,1e19,1)
tei0 = tei0[0]*norms["time"] # Ratio of time normalization to the temperature relaxation time

In [ ]:
def analyticDeltaT(x,*args):
    return 2*(args[0]**(3/2)-(1+x)**(3/2))/3+2*(np.sqrt(args[0])-np.sqrt(1+x)) + np.log((np.sqrt(args[0])-1)/(np.sqrt(args[0])+1)) - np.log((np.sqrt(1+x)-1)/(np.sqrt(1+x)+1)) - args[1]

In [ ]:
def analyticDeltaTJac(x,*args):

    return - (1+x)**(3/2)/x

In [ ]:
analyticDT = np.zeros(numFiles+1)
x0 = 1 + 0.5*(loadedData["T"].data[0,0] - loadedData["Ti"].data[0,0])/Ttot # initial condition (1+xi in Shkarofsky)
tei = np.zeros(numFiles+1)
for i in range(numFiles+1):
    tei[i] = tei0 * loadedData["time"].data[i] * loadedData["logLei"].data[i,0]
    guessdT = 0.5*(loadedData["T"].data[i,0] - loadedData["Ti"].data[i,0])/Ttot
    root = scipy.optimize.fsolve(analyticDeltaT,guessdT,args=(x0,tei[i]),fprime=analyticDeltaTJac)
    analyticDT[i] = root[0]*2*Ttot

In [ ]:
deltaT = loadedData["T"].data[:,0] - loadedData["Ti"].data[:,0]

In [ ]:
analyticCurve = hv.Curve((tei,analyticDT),label="Analytical formula")
numericalScatter = hv.Scatter((tei,deltaT),label="Numerical result").opts(marker="x",color="r",s=5.0)

overlay = analyticCurve*numericalScatter
overlay.opts(xlabel="$t'_{ei}$",ylabel="$\Delta T [10eV]$")

#### Visualising the kinetic over-relaxation

In [ ]:
error = deltaT-analyticDT

errorPlot=hv.Curve((tei,error)).opts(xlabel="$t'_{ei}$",ylabel="$\Delta T_{err} [10eV]$")

In [ ]:
hv.output(fig='pdf')
hv.save(overlay.opts(xlabel="$t'_{ei}$",ylabel="$\Delta T [10eV]$"), 'e-iTempRel.pdf', dpi=144)
hv.save(errorPlot,'e-iOverRel.pdf',dpi=144)